In [7]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import prompts as p
import json

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

PROMPT = p.dynamic_fewshot_multi_table
prompt_name = "dynamic_fewshot_multi_table"
MODEL = "gpt-4-turbo-preview"
TEMP = 0

DB_PEDIA_PATH = "../../Datasets/t2d_sm_nh/test/dbpedia_tables/{name}.csv"
WEBTABLES_PATH = "../../Datasets/t2d_sm_nh/test/webtables/{name}.csv"
OUTPUT_PATH = f"../../Datasets/Evaluations/Schema Matching/{prompt_name}_{MODEL}.csv"
CORRESPONDENCES_PATH = "../../Datasets/t2d_sm_nh/test/test_correspondences_sample_few_shot.csv"


In [8]:
# Read the CSV file into a DataFrame
df = pd.read_csv(CORRESPONDENCES_PATH)
df['y_pred'] = None
df['prompt_name'] = None
df['prompt'] = None
df['ai_answer'] = None

In [9]:
def extract_column(s):
    if "none" in s.lower() or "null" in s.lower():
        return -1
    else: 
        return int(s.split('-')[1])

def check_output(left, right, output):
    for x in output:
        if extract_column(x[0]) == left and extract_column(x[1]) == right:
            return True
    return False

In [10]:
def execute(df, prompt, prompt_name, start = 0):
    last = ""
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, timeout=10)
    for i in range(start, len(df)):
        row = df.iloc[i]
        if last != row['table_name']:
            last = row['table_name']
            if(row["y_pred"] != None and row["y_pred"] != "error"):
                continue
            print(i)
            chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

            print(df["table_name"][i])
            webtables_table =  row['table_A']
            db_pedia_table = row['table_B']
            example_1 = row['example_1']
            example_2 = row['example_2']
            sub_table = df[df['table_name'] == last]
            try: 
                result = chain.run(example_1 = example_1, example_2 = example_2, Table_A = webtables_table, Table_B = db_pedia_table, callbacks=[handler])
            except Exception as e:
                print(e)
                print(result)
                for j in sub_table.index:
                    df.at[j, 'y_pred'] =  'error'
                    df.at[j, 'prompt_name'] = prompt_name
                    df.at[j, 'prompt'] = chain.prompt.format_prompt(example_1 = example_1, example_2 = example_2, Table_A = webtables_table, Table_B = db_pedia_table).to_string()
                continue
            handler.langfuse.flush()
            prompt_text = chain.prompt.format_prompt(example_1 = example_1, example_2 = example_2, Table_A = webtables_table, Table_B = db_pedia_table).to_string()
            if (i < 5):
                print(prompt_text)
                print("\n")
                print(result)
                print("\n")
            
            if (i % 50 == 0):
                print(str(i) + " of  " + str(len(df)))
             
            try: 
                data = json.loads(result)
            except Exception as e:
                print("replacing Nones")
                result = result.replace('None', '"None"')
                data = json.loads(result)
            
            column_mappings = data["column_mappings"]
            
            for x in sub_table.index:
                y_pred = check_output(sub_table['column_index_left'][x], sub_table['column_index_right'][x], column_mappings)
                df.loc[x, 'y_pred'] =  y_pred
                df.loc[x, 'prompt_name'] = prompt_name
                df.loc[x, 'prompt'] = prompt_text
                df.loc[x, 'ai_answer'] = result
    return df

df = execute(df, PROMPT, prompt_name)
    



0
29021592_3_2299138476894681059
Human: Example 1: 
Input:
Table A: 
| Column A-0      |   Column A-1 | Column A-2     |   Column A-3 | Column A-4                   | Column A-5   | Column A-6   |
|:----------------|-------------:|:---------------|-------------:|:-----------------------------|:-------------|:-------------|
| Oceania         |          nan | nan            |       nan    | nan                          | nan          | nan          |
| Suriname        |           89 | 88;92.3,84.1   |        69.01 | 48Chr,27.4Hin,19.6Mus,5Sat   | $7,100       | 439,117      |
| Central America |          nan | nan            |       nan    | nan                          | nan          | nan          |
| Romania         |           94 | 98.4;99.1,97.7 |        71.63 | 86.8Ort,7.5Prot,4.7Cath,1Oth | $8,800       | 22,303,552   |
| San Marino      |          nan | 96;97,95       |        81.71 | Cath                         | $34,100      | 29,251       |

Table B: 
| Column B-0            

In [5]:
df.head(5)
print(df[df['y_pred'] == True])


     table_index                      table_name  column_index_left  \
0              0  29021592_3_2299138476894681059                  0   
5              5  29021592_3_2299138476894681059                  3   
12            12  34899692_0_6530393048033763438                  0   
23            23  34899692_0_6530393048033763438                  3   
36            36  55961337_0_6548713781034932742                  4   
67            67  14380604_4_3329235705746762392                  4   
90            90  66009064_0_9148652238372261251                  4   
100          100  84548468_0_5955155464119382182                  1   
134          134  99070098_0_2074872741302696997                  1   
145          145  86747932_0_7532457067740920052                  0   
149          149  48944826_0_2321751364268052533                  3   
158          158  48944826_0_2321751364268052533                  5   
161          161  48944826_0_2321751364268052533                  4   
173   

In [6]:
df.to_csv(OUTPUT_PATH, index = True)